# CNN 모델 평가

학습된 ChessCNN 모델을 평가합니다.

## 실행 순서
모든 셀을 순서대로 실행하면 됩니다.

## 평가 메트릭
- **Top-k Accuracy**: 정답이 상위 k개 예측 안에 포함되는 비율
- **Masked Accuracy**: 합법 수만 고려한 정확도
- **MRR (Mean Reciprocal Rank)**: 정답 순위의 역수 평균
- **Entropy**: 정책 분포의 불확실성
- **Legal Prob Mass**: 합법 수에 할당된 확률 질량

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pathlib import Path

from parquet_dataset import (
    ParquetChessDataset,
    get_parquet_file_info, 
    split_files_by_ratio
)
from chess_model import ChessCNN, load_model
from train_utils import validate

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

사용 장치: cuda
  GPU: NVIDIA GeForce RTX 5060 Ti
  GPU 메모리: 15.93 GB


## 모델 로드

In [3]:
# 모델 경로 설정
MODEL_DIR = Path("models")
BEST_MODEL_PATH = MODEL_DIR / "best_chess_cnn.pth"
LAST_MODEL_PATH = MODEL_DIR / "last_chess_cnn.pth"

# 평가할 모델 선택 (best 또는 last)
MODEL_PATH = BEST_MODEL_PATH  # 변경 가능: LAST_MODEL_PATH

# 모델 로드
model, checkpoint = load_model(MODEL_PATH, device=device)

print(f"로드된 모델 정보:")
print(f"  파일: {MODEL_PATH}")
print(f"  Epoch: {checkpoint.get('epoch', 'N/A')}")
print(f"  Validation Loss: {checkpoint.get('val_loss', 'N/A'):.4f}")
print(f"  Validation Accuracy: {checkpoint.get('val_accuracy', 'N/A')*100:.2f}%")
if 'train_loss' in checkpoint:
    print(f"  Train Loss: {checkpoint['train_loss']:.4f}")
if 'learning_rate' in checkpoint:
    print(f"  Learning Rate: {checkpoint['learning_rate']:.6f}")

로드된 모델 정보:
  파일: models/best_chess_cnn.pth
  Epoch: 5
  Validation Loss: 2.6870
  Validation Accuracy: 44.69%
  Train Loss: 2.6012
  Learning Rate: 0.008000


## 검증 데이터셋 로드

In [4]:
# 데이터 설정
PARQUET_DIR = "data/parquet"
TRAIN_RATIO = 0.9
BATCH_SIZE = 4096
CHUNK_SIZE = 32768

# 파일 정보 수집 및 분할 (val_files만 사용)
parquet_files, file_lengths = get_parquet_file_info(PARQUET_DIR, "chess_samples_*.parquet")
_, val_files, _, val_lengths, _, val_samples = split_files_by_ratio(
    parquet_files, 
    file_lengths, 
    train_ratio=TRAIN_RATIO,
    shuffle=False
)

# 검증 데이터셋 생성
print("\n검증 데이터셋 생성 중...")
val_dataset = ParquetChessDataset(
    parquet_files=val_files,
    file_lengths=val_lengths,
    shuffle_files=False,
    seed=42,
    batch_size=BATCH_SIZE,
    chunk_size=CHUNK_SIZE
)

print(f"검증 데이터: {val_dataset.estimated_length:,} 샘플")

VAL_STEPS = val_dataset.estimated_length // BATCH_SIZE
print(f"검증 이터레이션: {VAL_STEPS:,} steps")

# DataLoader (워커 없이 - 메모리 절약)
val_loader = DataLoader(
    val_dataset,
    batch_size=None,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)

print("\n데이터 로딩 준비 완료!")

## 모델 평가

In [5]:
# Loss 함수 및 설정
policy_loss_fn = nn.CrossEntropyLoss()
value_loss_fn = nn.MSELoss()
POLICY_WEIGHT = 1.0
VALUE_WEIGHT = 1.0
USE_AMP = torch.cuda.is_available()

# 평가 실행
print("="*60)
print("모델 평가 시작")
print("="*60)

metrics = validate(
    model, val_loader, policy_loss_fn, value_loss_fn,
    POLICY_WEIGHT, VALUE_WEIGHT, device,
    use_amp=USE_AMP, total_steps=VAL_STEPS,
)


In [6]:
print(f"\n" + "="*60)
print("평가 결과")
print("="*60)
print(f"\n📊 Loss:")
print(f"  Policy Loss: {metrics['policy_loss']:.4f}")
print(f"  Value Loss: {metrics['value_loss']:.4f}")
print(f"  Total Loss: {metrics['total_loss']:.4f}")

print(f"\n🎯 정확도 (마스크 미적용):")
print(f"  Top-1: {metrics['accuracy']*100:.2f}%")
print(f"  Top-3: {metrics['top3_acc']*100:.2f}%")
print(f"  Top-5: {metrics['top5_acc']*100:.2f}%")
print(f"  Top-10: {metrics['top10_acc']*100:.2f}%")

print(f"\n✅ 정확도 (마스크 적용):")
print(f"  Masked Top-1: {metrics['masked_acc']*100:.2f}%")

print(f"\n📈 Rank 분석:")
print(f"  MRR (Mean Reciprocal Rank): {metrics['mrr']:.4f}")
print(f"  Average Rank: {metrics['avg_rank']:.2f}")

print(f"\n📉 분포 분석:")
print(f"  Entropy (합법 수 기준): {metrics['entropy']:.4f}")
print(f"  Legal Prob Mass: {metrics['legal_prob_mass']*100:.2f}%")


평가 결과

📊 Loss:
  Policy Loss: 1.7733
  Value Loss: 0.9136
  Total Loss: 2.6870

🎯 정확도 (마스크 미적용):
  Top-1: 44.69%
  Top-3: 72.93%
  Top-5: 83.73%
  Top-10: 94.04%

✅ 정확도 (마스크 적용):
  Masked Top-1: 44.69%

📈 Rank 분석:
  MRR (Mean Reciprocal Rank): 0.6146
  Average Rank: 3.32

📉 분포 분석:
  Entropy (합법 수 기준): 1.6324
  Legal Prob Mass: 100.00%


## 샘플 예측 확인

In [ ]:
# 몇 개 샘플로 예측 확인
model.eval()
non_blocking = torch.cuda.is_available()

# 새 데이터셋 이터레이터 생성
sample_dataset = ParquetChessDataset(
    parquet_files=val_files[:1],  # 첫 파일만
    file_lengths=val_lengths[:1],
    shuffle_files=False,
    seed=42,
    batch_size=16,
    chunk_size=1024
)
sample_loader = DataLoader(sample_dataset, batch_size=None, num_workers=0)

with torch.no_grad():
    for states, policies, masks, values in sample_loader:
        states = states.to(device, non_blocking=non_blocking)
        masks = masks.to(device, non_blocking=non_blocking)
        
        policy_logits, value_pred = model(states, masks)
        policy_probs = torch.softmax(policy_logits, dim=1)
        pred_policies = policy_logits.argmax(dim=1)
        
        print("샘플 예측:")
        for i in range(min(5, len(states))):
            print(f"\n샘플 {i+1}:")
            print(f"  실제 Policy: {policies[i].item()}")
            print(f"  예측 Policy: {pred_policies[i].item()}")
            print(f"  정확도: {'✅' if pred_policies[i] == policies[i] else '❌'}")
            print(f"  실제 Value: {values[i].item():.4f}")
            print(f"  예측 Value: {value_pred[i].item():.4f}")
            print(f"  예측 확률 (top-1): {policy_probs[i].max().item():.4f}")
        
        break